In [1]:
import torch
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import pandas as pd

In [138]:
###  Load and Clean Data  ###

#Load as panda
xy_train = pd.read_csv('./data/titanic/train.csv')
x_test = pd.read_csv('./data/titanic/test.csv')
#xy_train = pd.read_csv('./data/titanic/debug_1s.csv')

#Scale Sex
xy_train['Sex'] = xy_train['Sex'].map({'male':1, 'female':0})
x_test['Sex'] = x_test['Sex'].map({'male':1, 'female':0})
#xy_train.Sex[xy_train.Sex == 'female'] = 1
#x_test.Sex[x_test.Sex == 'male'] = 0
#x_test.Sex[x_test.Sex == 'female'] = 1


#Scale Pclass
#xy_train.Pclass = xy_train.Pclass/3
#x_test.Pclass = x_test.Pclass/3 

#Scale Fare
#xy_train.Fare = xy_train.Fare/100
#x_test.Fare = x_test.Fare/100



xy_train.sample(3)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
475,476,0,1,"Clifford, Mr. George Quincy",1,NaN,0,0,110465,0.520,A14,S
834,835,0,3,"Allum, Mr. Owen George",1,18.0,0,0,2223,0.083,NaN,S
651,652,1,2,"Doling, Miss. Elsie",0,18.0,0,1,231919,0.230,NaN,S


In [102]:
#cleanup test
xy_train['Age'].fillna(xy_train['Age'].median(), inplace=True)
x_test['Age'].fillna(x_test['Age'].median(), inplace=True)

In [103]:
xy_train['FamilyS'] = xy_train['SibSp'] + xy_train['Parch'] + 1
x_test['FamilyS'] = x_test['SibSp'] + x_test['Parch'] + 1

def family(x):
    if x < 2:
        return 0/3
    elif x == 2:
        return 1/3
    elif x <= 4:
        return 2/3
    else:
        return 3/3
    
xy_train['FamilyS'] = xy_train['FamilyS'].apply(family)
x_test['FamilyS'] = x_test['FamilyS'].apply(family)

In [104]:
# get the number of missing data points per column
missing_values_count = xy_train.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count [5] #5 is age

# how many total missing values do we have?
total_cells = (xy_train.shape[0])
total_missing = missing_values_count[5]

# percent of data that is missing
(total_missing/total_cells) * 100

0.0

In [127]:
# encoding in dummy variable
pclass_dummies = pd.get_dummies(xy_train['Pclass'], prefix='Pclass')
xy_train = pd.concat([xy_train, pclass_dummies], axis=1)

In [128]:
xy_train.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Pclass_1,Pclass_2,Pclass_3
741,742,0,1,"Cavendish, Mr. Tyrell William",0,36.0,1,0,19877,0.788500,C46,S,1,0,0
329,330,1,1,"Hippach, Miss. Jean Gertrude",1,16.0,0,1,111361,0.579792,B18,C,1,0,0
712,713,1,1,"Taylor, Mr. Elmer Zebley",0,48.0,1,0,19996,0.520000,C126,S,1,0,0
732,733,0,2,"Knight, Mr. Robert J",0,NaN,0,0,239855,0.000000,NaN,S,0,1,0
241,242,1,3,"Murphy, Miss. Katherine ""Kate""",1,NaN,1,0,367230,0.155000,NaN,Q,0,0,1


In [112]:
class TitanicDataset(Dataset):
    """ Titanic dataset."""

    # Initialize your data, download, etc.
    def __init__(self):

        #Train data
        self.len = xy_train.shape[0]
        #self.pt_x_data = torch.from_numpy(np.array(xy_train.loc[:, ['PassengerId', 'Pclass', 'Sex']].values, dtype='float')).float()
        self.pt_x_data = torch.from_numpy(np.array(xy_train.loc[:, ['Sex', 'Parch', 'Age', 'FamilyS']].values, dtype='float')).float()
        #self.pt_x_data = torch.from_numpy(np.array(xy_train.loc[:, ['1','2','3']].values, dtype='float')).float()
        self.y_data = np.array(xy_train.loc[:, ['Survived']].values, dtype='float')
        self.pt_y_data = torch.from_numpy(self.y_data).float()
        
        #Test data
        self.pt_x_data_test = torch.from_numpy(np.array(x_test.loc[:, ['Sex', 'Parch', 'Age', 'FamilyS']].values, dtype='float')).float()
        #self.y_data_test = np.array(xy_test.loc[:, ['Survived']].values, dtype='float')

    def __getitem__(self, index):
        return self.pt_x_data[index], self.pt_y_data[index]

    def __len__(self):
        return self.len


dataset = TitanicDataset()

In [113]:
train_loader = DataLoader(dataset=dataset,
                          batch_size=50,
                          shuffle=True,
                          num_workers=0)

In [114]:
#type(train_loader)
print (dataset.pt_x_data.shape)
print (dataset.y_data.shape)
print (dataset.pt_y_data.shape)

print (dataset.pt_x_data)
print (dataset.y_data)


torch.Size([891, 4])
(891, 1)
torch.Size([891, 1])
tensor([[ 0.0000,  0.0000, 22.0000,  0.3333],
        [ 1.0000,  0.0000, 38.0000,  0.3333],
        [ 1.0000,  0.0000, 26.0000,  0.0000],
        ...,
        [ 1.0000,  2.0000, 28.0000,  0.6667],
        [ 0.0000,  0.0000, 26.0000,  0.0000],
        [ 0.0000,  0.0000, 32.0000,  0.0000]])
[[0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]

In [115]:
class Model(torch.nn.Module):

    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model, self).__init__()
        self.L1 = torch.nn.Linear(4, 5)
        self.L2 = torch.nn.Linear(5, 1)
        #self.l3 = torch.nn.Linear(4, 1)

        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        #out1 = self.sigmoid(self.l1(x))
        #out2 = self.sigmoid(self.l2(out1))
        #y_pred = self.sigmoid(self.l3(out2))
        
        out1 = self.relu(self.L1(x))
        y_pred = self.sigmoid(self.L2(out1))
        
        #y_pred = self.sigmoid(self.L1(x))
                
        return y_pred

# our model
model = Model()

In [120]:
# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='elementwise_mean' instead.
  warnings.warn(warning.format(ret))


In [121]:
# Training loop
for epoch in range(500):
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(inputs)

        # Compute and print loss
        loss = criterion(y_pred, labels)
        print(epoch, i, loss.data[0])

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app


0 0 tensor(0.6254)
0 1 tensor(0.7205)
0 2 tensor(0.6784)
0 3 tensor(0.6697)
0 4 tensor(0.6166)
0 5 tensor(0.6716)
0 6 tensor(0.6623)
0 7 tensor(0.7013)
0 8 tensor(0.5770)
0 9 tensor(0.6451)
0 10 tensor(0.6615)
0 11 tensor(0.7093)
0 12 tensor(0.6549)
0 13 tensor(0.6535)
0 14 tensor(0.6845)
0 15 tensor(0.6279)
0 16 tensor(0.6492)
0 17 tensor(0.6898)
1 0 tensor(0.6507)
1 1 tensor(0.6313)
1 2 tensor(0.6929)
1 3 tensor(0.6849)
1 4 tensor(0.6479)
1 5 tensor(0.6260)
1 6 tensor(0.6371)
1 7 tensor(0.6764)
1 8 tensor(0.6069)
1 9 tensor(0.6687)
1 10 tensor(0.6622)
1 11 tensor(0.6405)
1 12 tensor(0.6854)
1 13 tensor(0.6243)
1 14 tensor(0.7148)
1 15 tensor(0.7015)
1 16 tensor(0.6740)
1 17 tensor(0.6684)
2 0 tensor(0.5884)
2 1 tensor(0.7046)
2 2 tensor(0.6325)
2 3 tensor(0.6367)
2 4 tensor(0.6650)
2 5 tensor(0.6936)
2 6 tensor(0.6681)
2 7 tensor(0.6659)
2 8 tensor(0.6269)
2 9 tensor(0.6872)
2 10 tensor(0.6605)
2 11 tensor(0.6781)
2 12 tensor(0.6997)
2 13 tensor(0.6707)
2 14 tensor(0.6318)
2 15 tenso

27 16 tensor(0.7352)
27 17 tensor(0.6693)
28 0 tensor(0.6780)
28 1 tensor(0.6919)
28 2 tensor(0.6461)
28 3 tensor(0.6358)
28 4 tensor(0.6872)
28 5 tensor(0.6436)
28 6 tensor(0.6507)
28 7 tensor(0.6985)
28 8 tensor(0.6159)
28 9 tensor(0.6417)
28 10 tensor(0.7027)
28 11 tensor(0.6866)
28 12 tensor(0.6268)
28 13 tensor(0.6615)
28 14 tensor(0.6969)
28 15 tensor(0.6625)
28 16 tensor(0.5837)
28 17 tensor(0.6719)
29 0 tensor(0.7126)
29 1 tensor(0.6222)
29 2 tensor(0.7213)
29 3 tensor(0.6998)
29 4 tensor(0.6662)
29 5 tensor(0.6252)
29 6 tensor(0.6619)
29 7 tensor(0.6167)
29 8 tensor(0.6764)
29 9 tensor(0.6259)
29 10 tensor(0.6400)
29 11 tensor(0.6593)
29 12 tensor(0.6264)
29 13 tensor(0.7062)
29 14 tensor(0.6750)
29 15 tensor(0.6959)
29 16 tensor(0.6190)
29 17 tensor(0.6253)
30 0 tensor(0.6502)
30 1 tensor(0.6894)
30 2 tensor(0.6705)
30 3 tensor(0.6749)
30 4 tensor(0.6261)
30 5 tensor(0.6567)
30 6 tensor(0.6950)
30 7 tensor(0.7179)
30 8 tensor(0.6890)
30 9 tensor(0.6496)
30 10 tensor(0.6964)
3

57 5 tensor(0.6641)
57 6 tensor(0.6437)
57 7 tensor(0.6509)
57 8 tensor(0.6854)
57 9 tensor(0.6270)
57 10 tensor(0.6228)
57 11 tensor(0.6852)
57 12 tensor(0.6568)
57 13 tensor(0.6542)
57 14 tensor(0.6573)
57 15 tensor(0.5873)
57 16 tensor(0.7033)
57 17 tensor(0.7318)
58 0 tensor(0.6994)
58 1 tensor(0.6855)
58 2 tensor(0.6246)
58 3 tensor(0.6273)
58 4 tensor(0.6875)
58 5 tensor(0.7167)
58 6 tensor(0.5877)
58 7 tensor(0.6847)
58 8 tensor(0.6081)
58 9 tensor(0.7024)
58 10 tensor(0.6766)
58 11 tensor(0.5777)
58 12 tensor(0.6561)
58 13 tensor(0.6723)
58 14 tensor(0.6512)
58 15 tensor(0.7098)
58 16 tensor(0.6767)
58 17 tensor(0.6213)
59 0 tensor(0.6400)
59 1 tensor(0.6723)
59 2 tensor(0.6578)
59 3 tensor(0.6538)
59 4 tensor(0.6468)
59 5 tensor(0.6273)
59 6 tensor(0.7162)
59 7 tensor(0.6857)
59 8 tensor(0.6345)
59 9 tensor(0.7224)
59 10 tensor(0.6316)
59 11 tensor(0.6511)
59 12 tensor(0.6437)
59 13 tensor(0.6949)
59 14 tensor(0.6624)
59 15 tensor(0.6446)
59 16 tensor(0.6437)
59 17 tensor(0.63

82 0 tensor(0.6380)
82 1 tensor(0.6897)
82 2 tensor(0.6900)
82 3 tensor(0.6930)
82 4 tensor(0.6572)
82 5 tensor(0.6305)
82 6 tensor(0.6395)
82 7 tensor(0.6713)
82 8 tensor(0.6525)
82 9 tensor(0.6687)
82 10 tensor(0.6052)
82 11 tensor(0.6683)
82 12 tensor(0.6645)
82 13 tensor(0.6958)
82 14 tensor(0.6967)
82 15 tensor(0.6335)
82 16 tensor(0.6507)
82 17 tensor(0.6081)
83 0 tensor(0.6674)
83 1 tensor(0.6207)
83 2 tensor(0.6048)
83 3 tensor(0.7282)
83 4 tensor(0.6037)
83 5 tensor(0.6117)
83 6 tensor(0.6257)
83 7 tensor(0.6672)
83 8 tensor(0.6479)
83 9 tensor(0.6754)
83 10 tensor(0.6816)
83 11 tensor(0.6487)
83 12 tensor(0.6401)
83 13 tensor(0.7499)
83 14 tensor(0.6809)
83 15 tensor(0.6589)
83 16 tensor(0.7160)
83 17 tensor(0.6299)
84 0 tensor(0.6907)
84 1 tensor(0.5709)
84 2 tensor(0.6358)
84 3 tensor(0.6174)
84 4 tensor(0.6957)
84 5 tensor(0.6323)
84 6 tensor(0.6428)
84 7 tensor(0.6819)
84 8 tensor(0.6106)
84 9 tensor(0.6839)
84 10 tensor(0.7269)
84 11 tensor(0.6683)
84 12 tensor(0.6411)
8

...

397 2 tensor(0.6853)
397 3 tensor(0.6622)
397 4 tensor(0.6681)
397 5 tensor(0.6199)
397 6 tensor(0.6434)
397 7 tensor(0.6995)
397 8 tensor(0.6213)
397 9 tensor(0.6576)
397 10 tensor(0.7100)
397 11 tensor(0.5835)
397 12 tensor(0.5955)
397 13 tensor(0.6685)
397 14 tensor(0.6937)
397 15 tensor(0.5969)
397 16 tensor(0.6423)
397 17 tensor(0.6438)
398 0 tensor(0.6292)
398 1 tensor(0.6490)
398 2 tensor(0.6316)
398 3 tensor(0.6222)
398 4 tensor(0.6652)
398 5 tensor(0.6669)
398 6 tensor(0.7035)
398 7 tensor(0.6188)
398 8 tensor(0.6760)
398 9 tensor(0.5868)
398 10 tensor(0.7034)
398 11 tensor(0.5807)
398 12 tensor(0.7691)
398 13 tensor(0.6558)
398 14 tensor(0.6516)
398 15 tensor(0.6855)
398 16 tensor(0.5842)
398 17 tensor(0.6964)
399 0 tensor(0.6652)
399 1 tensor(0.5457)
399 2 tensor(0.6431)
399 3 tensor(0.6537)
399 4 tensor(0.6768)
399 5 tensor(0.6603)
399 6 tensor(0.6980)
399 7 tensor(0.6355)
399 8 tensor(0.6508)
399 9 tensor(0.7009)
399 10 tensor(0.6658)
399 11 tensor(0.6530)
399 12 tensor(0.

421 1 tensor(0.6545)
421 2 tensor(0.6407)
421 3 tensor(0.6290)
421 4 tensor(0.6146)
421 5 tensor(0.7019)
421 6 tensor(0.6881)
421 7 tensor(0.6968)
421 8 tensor(0.6597)
421 9 tensor(0.6715)
421 10 tensor(0.7062)
421 11 tensor(0.6063)
421 12 tensor(0.6305)
421 13 tensor(0.6653)
421 14 tensor(0.5876)
421 15 tensor(0.6534)
421 16 tensor(0.6785)
421 17 tensor(0.6682)
422 0 tensor(0.5558)
422 1 tensor(0.6342)
422 2 tensor(0.6217)
422 3 tensor(0.7198)
422 4 tensor(0.6378)
422 5 tensor(0.6196)
422 6 tensor(0.7100)
422 7 tensor(0.6820)
422 8 tensor(0.6565)
422 9 tensor(0.6405)
422 10 tensor(0.6860)
422 11 tensor(0.6892)
422 12 tensor(0.6488)
422 13 tensor(0.6056)
422 14 tensor(0.6839)
422 15 tensor(0.6406)
422 16 tensor(0.6363)
422 17 tensor(0.6901)
423 0 tensor(0.7183)
423 1 tensor(0.6235)
423 2 tensor(0.6507)
423 3 tensor(0.5875)
423 4 tensor(0.6567)
423 5 tensor(0.6533)
423 6 tensor(0.6138)
423 7 tensor(0.6462)
423 8 tensor(0.6442)
423 9 tensor(0.6875)
423 10 tensor(0.6622)
423 11 tensor(0.6

448 4 tensor(0.6902)
448 5 tensor(0.6398)
448 6 tensor(0.6728)
448 7 tensor(0.6739)
448 8 tensor(0.6835)
448 9 tensor(0.6448)
448 10 tensor(0.6726)
448 11 tensor(0.6975)
448 12 tensor(0.7175)
448 13 tensor(0.6827)
448 14 tensor(0.5995)
448 15 tensor(0.6069)
448 16 tensor(0.6641)
448 17 tensor(0.6273)
449 0 tensor(0.6838)
449 1 tensor(0.6347)
449 2 tensor(0.6555)
449 3 tensor(0.6442)
449 4 tensor(0.6303)
449 5 tensor(0.6540)
449 6 tensor(0.6142)
449 7 tensor(0.6497)
449 8 tensor(0.6656)
449 9 tensor(0.6998)
449 10 tensor(0.6189)
449 11 tensor(0.7049)
449 12 tensor(0.6741)
449 13 tensor(0.6510)
449 14 tensor(0.6441)
449 15 tensor(0.6741)
449 16 tensor(0.6958)
449 17 tensor(0.6805)
450 0 tensor(0.6836)
450 1 tensor(0.6460)
450 2 tensor(0.6724)
450 3 tensor(0.5965)
450 4 tensor(0.6027)
450 5 tensor(0.6839)
450 6 tensor(0.6406)
450 7 tensor(0.6741)
450 8 tensor(0.7199)
450 9 tensor(0.6349)
450 10 tensor(0.6640)
450 11 tensor(0.7035)
450 12 tensor(0.7047)
450 13 tensor(0.6546)
450 14 tensor(

472 4 tensor(0.6641)
472 5 tensor(0.6543)
472 6 tensor(0.7018)
472 7 tensor(0.6243)
472 8 tensor(0.6847)
472 9 tensor(0.6449)
472 10 tensor(0.6287)
472 11 tensor(0.6934)
472 12 tensor(0.6544)
472 13 tensor(0.6290)
472 14 tensor(0.7426)
472 15 tensor(0.6546)
472 16 tensor(0.5886)
472 17 tensor(0.5939)
473 0 tensor(0.6673)
473 1 tensor(0.6537)
473 2 tensor(0.6952)
473 3 tensor(0.6786)
473 4 tensor(0.6456)
473 5 tensor(0.6848)
473 6 tensor(0.6301)
473 7 tensor(0.6838)
473 8 tensor(0.6249)
473 9 tensor(0.6556)
473 10 tensor(0.6108)
473 11 tensor(0.6137)
473 12 tensor(0.6333)
473 13 tensor(0.6664)
473 14 tensor(0.6436)
473 15 tensor(0.7162)
473 16 tensor(0.6470)
473 17 tensor(0.7063)
474 0 tensor(0.6286)
474 1 tensor(0.7144)
474 2 tensor(0.6187)
474 3 tensor(0.7126)
474 4 tensor(0.5957)
474 5 tensor(0.7077)
474 6 tensor(0.6978)
474 7 tensor(0.6452)
474 8 tensor(0.6737)
474 9 tensor(0.6801)
474 10 tensor(0.6165)
474 11 tensor(0.5974)
474 12 tensor(0.6597)
474 13 tensor(0.6445)
474 14 tensor(

495 17 tensor(0.6327)
496 0 tensor(0.6773)
496 1 tensor(0.6960)
496 2 tensor(0.6542)
496 3 tensor(0.5806)
496 4 tensor(0.6464)
496 5 tensor(0.6641)
496 6 tensor(0.6402)
496 7 tensor(0.6743)
496 8 tensor(0.6540)
496 9 tensor(0.6392)
496 10 tensor(0.7150)
496 11 tensor(0.6441)
496 12 tensor(0.6540)
496 13 tensor(0.6920)
496 14 tensor(0.6701)
496 15 tensor(0.5886)
496 16 tensor(0.6501)
496 17 tensor(0.7105)
497 0 tensor(0.6440)
497 1 tensor(0.6589)
497 2 tensor(0.6710)
497 3 tensor(0.6246)
497 4 tensor(0.7243)
497 5 tensor(0.6543)
497 6 tensor(0.6660)
497 7 tensor(0.6375)
497 8 tensor(0.6279)
497 9 tensor(0.5940)
497 10 tensor(0.7053)
497 11 tensor(0.7048)
497 12 tensor(0.6740)
497 13 tensor(0.6585)
497 14 tensor(0.6517)
497 15 tensor(0.5849)
497 16 tensor(0.7455)
497 17 tensor(0.6077)
498 0 tensor(0.7545)
498 1 tensor(0.6350)
498 2 tensor(0.6837)
498 3 tensor(0.6682)
498 4 tensor(0.6544)
498 5 tensor(0.6381)
498 6 tensor(0.6142)
498 7 tensor(0.6809)
498 8 tensor(0.7001)
498 9 tensor(0.65

In [ ]:
#this will be last mini batch
print (y_pred.shape)

In [122]:
# Train and Test Set Predictions
print ("train set accuracy = " + str (accuracy(dataset.pt_x_data, dataset.y_data)))
#print ("test set accuracy = " + str (accuracy(dataset.pt_x_data_test, dataset.y_data_test)))

train set accuracy = 0.6285072951739619


In [ ]:
print (dataset.y_data_test)




In [123]:
#Run test set through
test_pred = model(dataset.pt_x_data_test)
print (test_pred.shape)
print (type(test_pred))

#Add prediction col to test panda
to_np = np.round(test_pred.detach().numpy())
to_np = to_np.astype(int)
#print (type(to_np))

print (x_test.shape)
x_test['Survived'] = to_np
print (x_test.columns)

#Export to csv for submission
x_test.to_csv('gender_parch_age_median_fams.csv', columns = ['PassengerId','Survived'], index = False)


torch.Size([418, 1])
<class 'torch.Tensor'>
(418, 12)
Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'FamilyS', 'Survived'],
      dtype='object')


In [71]:
def accuracy(X, Y):
    """
    This function is used to measure accuracy of the model.
    
    Arguments:
    X -- data set of examples you would like to label
    Y -- labels
    
    Returns:
    p -- predictions for the given dataset X
    """
    
    prediction = model(X)
    #print (prediction)
    np_prediction = np.round(prediction.detach().numpy())
    predictions_correct = np.equal(np_prediction, Y)
    #print (predictions_correct)
    p = np.mean(predictions_correct)
        
    return p

In [ ]:
......................Extra Stuff...........
convertfunc = lambda x: float(x.strip("%"))/100
test = np.genfromtxt('./data/titanic/train.csv', delimiter=',', skip_header=1, usecols = (0,1,2,4))

#aai = a.iloc[:, [0,3]].values
train_x = a.loc[:, ['PassengerId', 'Pclass', 'Sex']].values
train_y = a.loc[:, ['Survived']].values

In [ ]:
print ("pt_x_data: " + str (dataset.pt_x_data.shape))
print ("pt_y_data: " + str (dataset.pt_y_data.shape))
print ("y_data: " + str (dataset.y_data.shape))

In [ ]:
print (type (train_x))
print (type (train_y))

print (train_x)
print (train_y)

In [ ]:
#Read in debug file
debug_vals = np.genfromtxt('./data/titanic/debug_justPID.csv', delimiter=',', dtype=np.float32, skip_header=1)

#convert to correct format
#debug_x_data = torch.from_numpy(np.array(xy_train.loc[:, ['PassengerId', 'Pclass', 'Sex']].values, dtype='float')).float()
#debug_y_data = np.array(xy_train.loc[:, ['Survived']].values, dtype='float')

debug_x_data = torch.from_numpy(debug_vals[:, 0:-1])
debug_y_data = np.array(debug_vals[:, [-1]])
print (debug_x_data)
print (debug_y_data)

print ("debug set accuracy = " + str (accuracy(debug_x_data,debug_y_data)))

In [ ]:
a = np.array([10,-1])
b = np.array([10,1])



In [ ]:
np.dot(a.T,b)